In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vishpand\\Desktop\\projects\\e2e-mlops-project\\notebooks'

In [3]:
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\vishpand\\Desktop\\projects\\e2e-mlops-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
            ) -> None:
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [9]:
from pathlib import Path
from mlproject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config

    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets (0.75, 0.25) split
        train, test = train_test_split(data)
        train.to_csv(Path(self.config.root_dir) / "train.csv", index=False)
        test.to_csv(Path(self.config.root_dir) / "test.csv", index=False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(f"{train.shape = }")
        print(f"{test.shape = }")

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2024-07-04 22:38:31,442: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-04 22:38:31,445: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-04 22:38:31,449: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-04 22:38:31,451: INFO: common: created directory at: artifacts]
[2024-07-04 22:38:31,452: INFO: common: created directory at: artifacts/data_transformation]
[2024-07-04 22:38:31,484: INFO: 275030492: Splitted data into training and test sets]
[2024-07-04 22:38:31,485: INFO: 275030492: (1199, 12)]
[2024-07-04 22:38:31,486: INFO: 275030492: (400, 12)]
train.shape = (1199, 12)
test.shape = (400, 12)
